In [1]:
## Run this in jupyter so any changes you make to the class will appear just by re-importing
## instead of having to re-start the kernel
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
from electrical_model import ElectricModel

In [4]:
### read in commuter_df. feed that and the ev file location to class
commuter_df = pd.read_csv("./samples/assignment_00_random.csv",index_col=0)
em_1 = ElectricModel(commuter_df,ev_reference_table_loc="EV_reference_table.csv"\
                    ,PEV_delay_hr=3,bus_ferry_cab_delay=6)
em_1

In [5]:
print(em_1.commuter_df.shape)

(23978, 43)


In [6]:
### read_electric_info_in reads ev_ref_table and creates floor variables
em_1.read_electric_info_in()
print(f"Subway floor kwh: {em_1.subway_floor} || Commuter Rail floor kwh: {em_1.commuterRail_floor}")
em_1.ev_reference_table.head(2)

ev reference table read in. subway floor and commuter rail floor extracted from table
Subway floor kwh: 55240 || Commuter Rail floor kwh: 9147


,TransMode,ChargingPower_kW,HoursToFullyRecharge_hr,Floor_kWh,Multiplier,MultiplierUnit,excludingfromMNY,NumberOfEV,TotalEnergyPerEV,HourEnergyPerEV,Energy
0,AutoOccupants,12.0,7.0,NaN,0.2134,kwh/km,Live outside of MNY & Owner,PERWT,DISTANCE_KM*Multiplier*2,no delay + related to charging time,In
1,Bus,163.0,3.0,NaN,1.7610,kwh/km,Live outside of MNY,PERWT/20,DISTANCE_KM*Multiplier,6 hr delay + related to charging time,In+Out


In [7]:
## clean_commuter_df:
    # creates the MNY_RES column
    # renames the Rand Assignment to TransMode (if necessary)
    # Creates leave_work_hour column
    # Filters out reverse commuters
    # runs excludingfromMNY function

em_1.clean_commuter_df()
print(em_1.commuter_df.shape)

(18898, 46)


In [8]:
commuter_df.head(5)

,YEAR,PERWT,HOME_STATEFIP,HOME_PUMA,PUMA_NAME,SEX,AGE,HRS_WK_DAILY,TOTAL_PERSONAL_INCOME,MODE_TRANSP_TO_WORK,...,FLAG_EBUSES,FLAG_SUBWAY,FLAG_COMMUTERRAIL,FLAG_FERRY,FLAG_ESCOOTER,FLAG_WALK,FLAG_EBIKE,FLAG_WFH,Rand Assignment,MNY_RES
0,2019,23.0,9,500,Litchfield County,F,46,7.4,3800,"Auto, truck, or van",...,0,0,0,0,0,0,0,0,No option,0
1,2019,48.0,9,500,Litchfield County,M,51,11.0,488000,"Auto, truck, or van",...,0,0,0,0,0,0,0,0,Motorcycle,0
2,2019,54.0,9,500,Litchfield County,F,44,11.0,438000,"Auto, truck, or van",...,0,0,0,0,0,0,0,0,Motorcycle,0
3,2019,29.0,9,500,Litchfield County,M,28,10.0,20000,"Auto, truck, or van",...,0,0,0,0,0,0,0,0,No option,0
4,2019,33.0,9,500,Litchfield County,M,66,12.0,727000,"Auto, truck, or van",...,0,0,0,0,0,0,0,0,Motorcycle,0


In [9]:
em_1.create_in_out_flow_dfs()
em_1.flow_df.head(5)

,HOUR,HRS_WK_DAILY,TransMode,DISTANCE_KM,PERWT,FLOW_DIR
0,9,8,No option,170.50491,23.0,IN
1,8,11,Motorcycle,170.50491,48.0,IN
2,8,11,Motorcycle,170.50491,54.0,IN
3,14,10,No option,170.50491,29.0,IN
4,9,12,Motorcycle,170.50491,33.0,IN


In [10]:
# aggregate_data() runs HourEnergyByMode wrapper function
em_1.aggregate_data()

In [11]:
em_1.df_aggregate

### this looks almost identical to the previous notebook's df_agg
### slight differences could be due to the random function 
    # filtering different bicycle and escooter home owners

,Charge_Hour,TransMode,PEV_DELAY,FLOW_DIR,Energy
0,0,AutoOccupants,Earliest,IN,90.988519
1,0,AutoOccupants,Random,IN,156.702450
2,0,Bicycle,Custom,IN,1508.385451
3,0,Bicycle,Earliest,IN,5393.925523
4,0,Bicycle,Latest,IN,2660.603086
...,...,...,...,...,...
1255,23,Escooter,NaN,ALL,150.346712
1256,23,Ferry,NaN,ALL,226.400000
1257,23,Motorcycle,NaN,ALL,64027.583455
1258,23,Subway,NaN,ALL,463355.128000


In [12]:
#### Checks from Jingrong's notebook
print(em_1.df_aggregate.TransMode.unique())
print()
print(em_1.df_aggregate.FLOW_DIR.unique())

['AutoOccupants', 'Bicycle', 'Bus', 'CommuterRail', 'Escooter', 'Ferry', 'Motorcycle', 'Subway', 'Taxicab']
Categories (9, object): ['AutoOccupants' < 'Bus' < 'Ferry' < 'Escooter' ... 'Motorcycle' < 'Taxicab' < 'Subway' < 'CommuterRail']

['IN' 'OUT' 'ALL']


In [13]:
em_1.df_aggregate.groupby(by=["PEV_DELAY"]).agg({"Energy":"sum"}).reset_index() 
### should be the same

,PEV_DELAY,Energy
0,Custom,1.005985e+07
1,Earliest,1.005985e+07
2,Latest,1.005985e+07
3,Random,1.005985e+07


In [14]:
## ToDo:
# Create visualization methods 
# Create diagnostic methods that can spit out certain values on demand
# Create input for electrical capacity parameters (once Robert gives us info)